# 🚀 NEURAL RISK v0.2.0 - LIVE DASHBOARD

**Real-time monitoring del sistema**
- Refresca cada 30 segundos
- PnL en vivo
- Posiciones activas
- Métricas por experto
- Alertas de anomalías

In [ ]:
import pandas as pd
import numpy as np
import sqlite3
import yaml
from datetime import datetime, timedelta
import time
from IPython.display import clear_output, HTML
import plotly.graph_objects as go
import plotly.express as px

# Setup
with open('config/config.yaml') as f:
    config = yaml.safe_load(f)

db_path = config['database']['path']

print("✅ Dashboard initialized")

## 📊 PnL REAL-TIME

In [ ]:
def get_pnl_metrics():
    """Obtiene métricas de PnL actual"""
    try:
        conn = sqlite3.connect(db_path)
        
        # PnL total
        query_total = '''
            SELECT 
                COALESCE(SUM(pnl), 0) as total_pnl,
                COUNT(*) as n_trades
            FROM fills
        '''
        df_total = pd.read_sql(query_total, conn)
        
        # PnL diario
        query_daily = '''
            SELECT 
                COALESCE(SUM(pnl), 0) as daily_pnl
            FROM fills
            WHERE date(timestamp) = date('now')
        '''
        df_daily = pd.read_sql(query_daily, conn)
        
        # PnL por asset (últimas 24h)
        query_asset = '''
            SELECT asset, SUM(pnl) as pnl, COUNT(*) as trades
            FROM fills
            WHERE timestamp > datetime('now', '-24 hours')
            GROUP BY asset
            ORDER BY pnl DESC
        '''
        df_asset = pd.read_sql(query_asset, conn)
        
        conn.close()
        
        return {
            'total_pnl': df_total['total_pnl'].iloc[0] if len(df_total) > 0 else 0,
            'n_trades': df_total['n_trades'].iloc[0] if len(df_total) > 0 else 0,
            'daily_pnl': df_daily['daily_pnl'].iloc[0] if len(df_daily) > 0 else 0,
            'by_asset': df_asset
        }
    except Exception as e:
        print(f"Error: {e}")
        return None

metrics = get_pnl_metrics()
if metrics:
    print(f"""\n╔════════════════════════════════════════╗
║         💰 PnL METRICS                 ║
╠════════════════════════════════════════╣
║ Total PnL:        {metrics['total_pnl']:>15,.2f} USDT ║
║ Daily PnL:        {metrics['daily_pnl']:>15,.2f} USDT ║
║ Total Trades:     {metrics['n_trades']:>15} ║
╚════════════════════════════════════════╝""")
    
    if len(metrics['by_asset']) > 0:
        print("\n📈 PnL por Asset (últimas 24h):")
        print(metrics['by_asset'].to_string(index=False))

## 📍 POSICIONES ACTIVAS

In [ ]:
def get_open_positions():
    """Obtiene posiciones abiertas actuales"""
    try:
        conn = sqlite3.connect(db_path)
        
        query = '''
            SELECT 
                o.asset,
                o.side,
                o.quantity,
                o.price as entry_price,
                d.stop_loss,
                (SELECT price FROM market_data WHERE asset = o.asset ORDER BY timestamp DESC LIMIT 1) as current_price,
                d.confidence,
                d.signal,
                MAX(o.timestamp) as entry_time
            FROM orders o
            LEFT JOIN engine_decisions d ON o.asset = d.asset
            WHERE o.status = 'FILLED'
            AND NOT EXISTS (
                SELECT 1 FROM fills f WHERE f.order_id = o.order_id
            )
            GROUP BY o.asset, o.side
        '''
        
        df = pd.read_sql(query, conn)
        conn.close()
        
        if len(df) > 0:
            df['unrealized_pnl'] = (
                (df['current_price'] - df['entry_price']) * df['quantity'] * 
                np.where(df['side'] == 'BUY', 1, -1)
            )
            df['unrealized_pnl_pct'] = (
                (df['current_price'] - df['entry_price']) / df['entry_price'] * 100 * 
                np.where(df['side'] == 'BUY', 1, -1)
            )
        
        return df
    except Exception as e:
        print(f"Error: {e}")
        return pd.DataFrame()

positions = get_open_positions()

if len(positions) > 0:
    print("\n📍 POSICIONES ABIERTAS:")
    print("-" * 100)
    for idx, row in positions.iterrows():
        pnl_color = "🟢" if row['unrealized_pnl'] > 0 else "🔴"
        print(f"{pnl_color} {row['asset']:5} | {row['side']:4} | {row['quantity']:.4f} | "
              f"Entry: {row['entry_price']:.2f} | Current: {row['current_price']:.2f} | "
              f"PnL: {row['unrealized_pnl']:,.2f} ({row['unrealized_pnl_pct']:+.2f}%) | "
              f"SL: {row['stop_loss']:.2f}")
else:
    print("\nℹ️  No posiciones activas")

## 🧠 MÉTRICAS POR EXPERTO

In [ ]:
def get_expert_performance():
    """Calcula performance por experto (desde decisiones guardadas)"""
    try:
        conn = sqlite3.connect(db_path)
        
        query = '''
            SELECT 
                signal,
                COUNT(*) as signal_count,
                AVG(confidence) as avg_confidence,
                AVG(agreement) as avg_agreement
            FROM engine_decisions
            WHERE timestamp > datetime('now', '-24 hours')
            GROUP BY signal
        '''
        
        df = pd.read_sql(query, conn)
        conn.close()
        
        return df
    except Exception as e:
        print(f"Error: {e}")
        return pd.DataFrame()

expert_perf = get_expert_performance()

print("\n🧠 DESEMPEÑO POR TIPO DE SEÑAL (últimas 24h):")
print("-" * 80)

if len(expert_perf) > 0:
    for idx, row in expert_perf.iterrows():
        signal_icon = "📈" if row['signal'] == 'LONG' else "📉" if row['signal'] == 'SHORT' else "⏸️ "
        print(f"{signal_icon} {row['signal']:6} | Señales: {row['signal_count']:3} | "
              f"Confianza: {row['avg_confidence']:.2f} | Agreement: {row['avg_agreement']:+.2f}")
else:
    print("No data available")

## ⚠️ ALERTAS & ANOMALÍAS

In [ ]:
def check_alerts():
    """Chequea condiciones de alerta"""
    alerts = []
    
    # Alert 1: Daily loss
    metrics = get_pnl_metrics()
    if metrics and metrics['daily_pnl'] < -1000:
        alerts.append(f"🔴 ALERTA: Pérdida diaria > USDT {abs(metrics['daily_pnl']):,.0f}")
    
    # Alert 2: No trades
    if metrics and metrics['n_trades'] == 0:
        alerts.append(f"🟡 ALERTA: Sin trades ejecutados aún")
    
    # Alert 3: Models stale (> 24h)
    if os.path.exists('./data/trained_models.pkl'):
        model_age = (datetime.now() - datetime.fromtimestamp(os.path.getmtime('./data/trained_models.pkl'))).total_seconds()
        if model_age > 86400:
            alerts.append(f"🟡 ALERTA: Modelos no actualizados hace {model_age/3600:.1f} horas")
    
    return alerts

alerts = check_alerts()
if alerts:
    print("\n⚠️  ALERTAS:")
    for alert in alerts:
        print(f"  {alert}")
else:
    print("\n✅ Sin alertas")

## 🔄 AUTO-REFRESH (Cada 30s)

In [ ]:
import os

# Ejecuta loop de refresco
print("🚀 Iniciando auto-refresh (Ctrl+C para parar)...\n")

try:
    cycle = 0
    while True:
        clear_output(wait=True)
        
        cycle += 1
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
        print(f"╔════════════════════════════════════════════════════════════════╗")
        print(f"║  🚀 NEURAL RISK v0.2.0 LIVE DASHBOARD - Ciclo {cycle:3}          ║")
        print(f"║  {timestamp}                            ║")
        print(f"╚════════════════════════════════════════════════════════════════╝\n")
        
        # 1. PnL
        metrics = get_pnl_metrics()
        if metrics:
            color_total = "🟢" if metrics['total_pnl'] > 0 else "🔴"
            color_daily = "🟢" if metrics['daily_pnl'] > 0 else "🔴"
            
            print(f"""╔════════════════════════════════════════╗
║         💰 PnL METRICS                 ║
╠════════════════════════════════════════╣
║ {color_total} Total PnL:        {metrics['total_pnl']:>13,.2f} USDT ║
║ {color_daily} Daily PnL:        {metrics['daily_pnl']:>13,.2f} USDT ║
║ Total Trades:     {metrics['n_trades']:>15} ║
╚════════════════════════════════════════╝\n""")
        
        # 2. Posiciones
        positions = get_open_positions()
        if len(positions) > 0:
            print("📍 POSICIONES ABIERTAS:")
            for idx, row in positions.iterrows():
                pnl_color = "🟢" if row['unrealized_pnl'] > 0 else "🔴"
                print(f"  {pnl_color} {row['asset']:5} | {row['side']:4} | {row['quantity']:.4f} | "
                      f"PnL: {row['unrealized_pnl']:,.2f} ({row['unrealized_pnl_pct']:+.2f}%)")
            print()
        else:
            print("📍 POSICIONES: Ninguna abierta\n")
        
        # 3. Expertos
        expert_perf = get_expert_performance()
        if len(expert_perf) > 0:
            print("🧠 SEÑALES (últimas 24h):")
            for idx, row in expert_perf.iterrows():
                signal_icon = "📈" if row['signal'] == 'LONG' else "📉" if row['signal'] == 'SHORT' else "⏸️ "
                print(f"  {signal_icon} {row['signal']:6} | {row['signal_count']:3} | "
                      f"Conf: {row['avg_confidence']:.2f} | Agree: {row['avg_agreement']:+.2f}")
            print()
        
        # 4. Alertas
        alerts = check_alerts()
        if alerts:
            print("⚠️  ALERTAS:")
            for alert in alerts:
                print(f"  {alert}")
        else:
            print("✅ Sin alertas")
        
        print(f"\n⏳ Siguiente refresco en 30s... (Ctrl+C para parar)")
        
        time.sleep(30)

except KeyboardInterrupt:
    print("\n\n🛑 Dashboard parado")